In [16]:
# ! pip install bs4 selenium webdriver-manager -q

In [17]:
import pandas as pd
import sys

sys.path.append("..")

from src.clients.yc_client import YCClient
from src.clients.perplexity_client import PerplexityClient

### Scraping (www.ycombinator.com/companies)

In [18]:
yc = YCClient()

batch_codes = []

# range [i, j] is 200i to 200j.  W and S.
for year in range(21, 17, -1):
    yr = str(year).zfill(2)
    batch_codes.extend([f"W{yr}", f"S{yr}"])


output_dir = "../data/live/yc"

# df = yc.scrape_batches(batch_codes, output_dir=None)

yc.__del__()

In [19]:
# df

### Get exits and funding for companies

Some of these numbers, especially exits are off. But this data should get close enough to map to ordinals and success.

In [ ]:
pc = PerplexityClient()

for bc in batch_codes:
    try:
        filename = f'{output_dir}/{bc}.csv'
        
        batch_df = pd.read_csv(filename)
        batch_df = batch_df.drop_duplicates(subset=['Name', 'LinkedIn'])
        batch_df = batch_df.dropna(subset=['Name', 'LinkedIn'])
        
        for company in batch_df['Company'].unique():
            try:
                evaluation = pc.eval_company(f'{company} (YC {bc})')
                
                batch_df.loc[batch_df['Company'] == company, 'exit_value_usd'] = evaluation.get('exit_value_usd')
                batch_df.loc[batch_df['Company'] == company, 'total_funding_usd'] = evaluation.get('total_funding_usd')
                
                batch_df.to_csv(filename, index=False)
                print(f"Processed {company}")
                
            except Exception as e:
                print(f"Error evaluating {company} (YC {bc}): {e}")
        
    except Exception as e:
        print(f"Error loading {bc}: {e}")